<a href="https://colab.research.google.com/github/shreyasudaya/compvismask/blob/main/Mask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
import numpy as np
from matplotlib import pyplot as plt
import os
import cv2
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
!unzip -uq "/content/drive/MyDrive/Facemask/archive" -d "/content/drive/MyDrive/Facemask/archive"


Mounted at /content/drive


In [ ]:
main_dir="/content/drive/MyDrive/Facemask/archive/New Masks Dataset"
train_dir = os.path.join(main_dir,'Train')
test_dir = os.path.join(main_dir,'Test')
valid_dir = os.path.join(main_dir,'Validation')
train_mask_dir=os.path.join(train_dir,'Mask')
train_nomask_dir=os.path.join(train_dir,'Non Mask')
train_mask_names= os.listdir(train_mask_dir)
train_nomask_names= os.listdir(train_nomask_dir)
train_datagen=ImageDataGenerator(rescale=1./255,
                                 zoom_range=0.2,
                                 rotation_range=40,
                                 horizontal_flip= True
                                 )
test_datagen=ImageDataGenerator(rescale=1./255)
validation_datagen=ImageDataGenerator(rescale=1./255)
train_generator=train_datagen.flow_from_directory(train_dir,
                                                  target_size= (150,150),
                                                  batch_size= 32,
                                                  class_mode= 'binary')
test_generator=test_datagen.flow_from_directory(test_dir,
                                                  target_size=(150,150),
                                                  batch_size=32,
                                                  class_mode='binary')
valid_generator=validation_datagen.flow_from_directory(valid_dir,
                                                  target_size=(150,150),
                                                  batch_size=32,
                                                  class_mode='binary')
#Building a sequential model with two layers
model=Sequential()
model.add(Conv2D(32,(3,3),padding='SAME',activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(64,(3,3),padding='SAME',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Found 700 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Found 306 images belonging to 2 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 75, 75, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             

In [ ]:
#model is configured with optimizer for gradient descent and lossfunction for binary classification
#training the model with around 50 epochs.
model.compile(optimizer='adam',loss='binary_crossentropy', metrics =['accuracy'])
history = model.fit(train_generator,epochs=50,validation_data=valid_generator)
test_loss,test_acc = model.evaluate(test_generator)

Epoch 1/50
22/22 [==============================] - 372s 17s/step - loss: 2.4820 - accuracy: 0.4900 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/50
 2/22 [=>............................] - ETA: 40s - loss: 0.6973 - accuracy: 0.5156 

KeyboardInterrupt: ignored

In [ ]:

from google.colab.patches import cv2_imshow
model=load_model('/content/drive/MyDrive/Facemask/saved_model.h5')

img_width,img_height=150,150

face_cascade=cv2.CascadeClassifier('/content/drive/MyDrive/Facemask/haarcascade_frontalface_default.xml')

cap=cv2.VideoCapture('/content/drive/MyDrive/Facemask/mask_video.mp4')

img_count_full=0

font=cv2.FONT_HERSHEY_SIMPLEX
org=(1,1)
class_label=''
fontScale=1
color=(255,0,0)
thickness=2

while True:
   img_count_full += 1
   response, color_img = cap.read()

   if response == False:
       break
   scale = 50
   width=int(color_img.shape[1]*scale/100)
   height=int(color_img.shape[0]*scale/100)
   dim=(width,height)

   color_img= cv2.resize(color_img, dim , interpolation=cv2.INTER_AREA)
  
   gray_img= cv2.cvtColor(color_img,cv2.COLOR_BGR2GRAY)
  
   faces = face_cascade.detectMultiScale(gray_img, 1.1, 6)
  
   img_count=0
   for(x,y,w,h) in faces:
      org=(x-10,y-10)
      img_count+=1
      color_face=color_img[y:y+h,x:x+w]
      cv2.imwrite('/content/%d%dface.jpg'%(img_count_full,img_count),color_face)
      img = load_img('/content/%d%dface.jpg'%(img_count_full,img_count),target_size=(img_width,img_height))
      img = img_to_array(img)
      img = np.expand_dims(img,axis=0)
      prediction = model.predict(img)


      if prediction == 0:
          class_label="Mask"
          color=(0,255,0)
      else:
          class_label="No Mask"
          color=(0,0,255)
    
      cv2.rectangle(color_img,(x,y),(x+w,y+h),(255,0,0),3)
      cv2.putText(color_img, class_label, org, font, fontScale, color, thickness, cv2.LINE_AA)

   cv2_imshow(color_img)
   if cv2.waitKey(1) & 0xFF == ord('q'):
       break
cap.release()
cv2.destroyAllWindows()

NameError: ignored